# 0. Zeroth Law, load all the datasets!

In [ ]:
import pandas as pd

df_dominant = pd.read_csv('datasett/train.csv')
df_breed = pd.read_csv('datasett/BreedLabels.csv')
df_color = pd.read_csv('datasett/ColorLabels.csv')
df_state = pd.read_csv('datasett/StateLabels.csv')

df_dominant

# 1. Data Preprocessing by replacing numbers to attribute

- PetID - Unique hash ID of pet profile
- AdoptionSpeed - Categorical speed of adoption. Lower is faster. This is the value to predict. See below section for more info.
- Type - Type of animal (1 = Dog, 2 = Cat)
- Name - Name of pet (Empty if not named)
- Age - Age of pet when listed, in months
- Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
- Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)
- Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
- Color1 - Color 1 of pet (Refer to ColorLabels dictionary)
- Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
- Color3 - Color 3 of pet (Refer to ColorLabels dictionary)
- MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
- FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
- Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
- Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
- Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
- Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
- Quantity - Number of pets represented in profile
- Fee - Adoption fee (0 = Free)
- State - State location in Malaysia (Refer to StateLabels dictionary)
- RescuerID - Unique hash ID of rescuer
- VideoAmt - Total uploaded videos for this pet
- PhotoAmt - Total uploaded photos for this pet
- Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.

In [19]:
df_processed = df_dominant

Merging State from df_state

In [ ]:
df_processed = df_processed.merge(df_state, left_on='State', right_on='StateID', how='left') # Bring the StateName from df_
df_processed

In [ ]:
# rename stateID to state
df_processed = df_processed.drop(columns=['State','StateID'])
df_processed = df_processed.rename(columns={'StateName':'State'})
df_processed

Merging breed from df_breed


In [ ]:
df_breed

There is 2 columns for breed, which is breed1 and breed2. 
If breed2 is zero, it means the animal is pure blooded, while if breed2 is a number, the animal is mixed with 2 types of breed.

For pureblooded, i will copy the breed1 to breed2.

In [23]:
df_processed.loc[df_processed['Breed2'] == 0, 'Breed2'] = df_processed['Breed1']

In [ ]:
# get data from df_breed to df_processed
df_processed = df_processed.merge(df_breed, left_on=['Type','Breed1'], right_on=['Type','BreedID'], how='left')
df_processed

In [ ]:
# get data from df_breed to df_processed but for breed2
df_processed = df_processed.merge(df_breed, left_on=['Type','Breed2'], right_on=['Type','BreedID'], how='left')
df_processed

- Type - Type of animal (1 = Dog, 2 = Cat)
- Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
- MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
- FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
- Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
- Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
- Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
- Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)

In [ ]:
df_processed.columns

In [29]:
df_processed['Type'] = df_processed['Type'].replace({1:'Dog',2:'Cat'})
df_processed['Gender'] = df_processed['Gender'].replace({1:'Male',2:'Female',3:'Mixed'})
df_processed['MaturitySize'] = df_processed['MaturitySize'].replace({1:'Small',2:'Medium',3:'Large',4:'Extra Large',0:'NA'})
df_processed['FurLength'] = df_processed['FurLength'].replace({1:'Short',2:'Medium',3:'Long',0:'NA'})
df_processed['Vaccinated'] = df_processed['Vaccinated'].replace({1:'Yes',2:'No',3:'NA'})
df_processed['Dewormed'] = df_processed['Dewormed'].replace({1:'Yes',2:'No',3:'NA'})
df_processed['Sterilized'] = df_processed['Sterilized'].replace({1:'Yes',2:'No',3:'NA'})
df_processed['Health'] = df_processed['Health'].replace({1:'Healthy',2:'Minor Injury',3:'Serious Injury',0:'NA'})

In [ ]:
df_processed

There is 3 columns for color, which is color1 color2 color3

if color 2 || color 3 is 0, it means the animal is pure colored
therefore i will copy besides the color

In [ ]:
color_mapping = {1:'Black',2:'Brown',3:'Golden',4:'Yellow',5:'Cream',6:'Gray',7:'White'}

df_processed[['Color1','Color2','Color3']] = df_processed[['Color1','Color2','Color3']].replace(color_mapping)
df_processed

In [ ]:
# check for null
zero_count = (df_processed['Color1'] == 0).sum()
print(zero_count)
zero_count = (df_processed['Color2'] == 0).sum()
print(zero_count)
zero_count = (df_processed['Color3'] == 0).sum()
print(zero_count)

In [ ]:
# apply color1 to color2
df_processed['Color2'] = df_processed.apply(lambda row: row['Color1'] if row['Color2'] == 0 else row['Color2'], axis=1)
# apply color2 to color3
df_processed['Color3'] = df_processed.apply(lambda row: row['Color2'] if row['Color3'] == 0 else row['Color3'], axis=1)
df_processed

Data transfer should all be settled!

# 2. Data Preprocessing for Null

In [ ]:
df_processed.isnull().sum()

- Name shouldn't be a big problem, so I would just put a generic name.
- The description, breedid x and y is a small portion of the entire thing (14k entry), so easy drop


In [41]:
df_processed['Name'] = df_processed['Name'].fillna('Still Water')

In [42]:
df_processed = df_processed.dropna(subset=['BreedID_x','Description','BreedID_y'])

In [ ]:
df_processed.isnull().sum()

In [ ]:
df_processed

In [45]:
import os
os.makedirs('output',exist_ok=True)
df_processed.to_csv('output/test.csv',index=True)

Now that we have import and merge the numbers, we should drop it
I am dropping
- Breed1
- Breed2
- RescuerID
- Description
- BreedID_x
- BreedID_y

In [46]:
df_processed = df_processed.drop(columns=['Breed1', 'Breed2','RescuerID','Description','BreedID_x','BreedID_y'])

In [ ]:
df_processed

# 3. Formatting and Beautifying

Rearrange column position!

In [48]:
df_processed = df_processed[['PetID','Type','Name','Age','BreedName_x','BreedName_y','Gender','Color1','Color2','Color3','MaturitySize','FurLength','Vaccinated','Dewormed','Sterilized','Health','State','Quantity','Fee','VideoAmt','PhotoAmt','AdoptionSpeed']]

In [ ]:
df_processed

Change BreedName x and y to Breed1 and Breed2

In [ ]:
df_processed = df_processed.rename(columns={'BreedName_x':'Breed1','BreedName_y':'Breed2'})
df_processed

Make 0,1,2 as fast while 3,4 slow in adoption speed

In [ ]:
df_processed['AdoptionSpeedBool'] = df_processed['AdoptionSpeed'].replace({0:'Fast',1:'Fast',2:'Fast',3:'Slow',4:'Slow'})
df_processed

check if anything is wrong

In [ ]:
df_processed.isnull().sum()

# 4. Export the data to a csv

In [ ]:
import os
os.makedirs('datasett', exist_ok=True)
df_processed.to_csv('datasett/meow_cleaned.csv',index=True)
print("Exported to datasett/meow_cleaned.csv!")